## Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# The dataframe is relatively large and we want to visualise it as a scrollable element
pd.set_option('display.max_columns', None)

In [3]:
# Import the cleaned data
df = pd.read_csv('data/cleaned_data.csv')

# Import training data
df_train = pd.read_csv("data/train.csv", index_col=0)

# Import test data
df_test = pd.read_csv("data/test.csv", index_col=0)

# Import forecast data
df_forecast = pd.read_csv("data/forecast.csv", index_col=0)

In [4]:
# Check if succeeded
print(df.shape)
print(df_test.shape)
print(df_forecast.shape)
print(df_train.shape)

(818375, 22)
(5600, 3)
(5504, 3)
(93800, 3)


In [5]:
# Only adjust the copy version
data = df.copy()

In [6]:
data.shape

(818375, 22)

In [7]:
# Parse timestamp
data["DimPostingDateId"] = pd.to_datetime(data["DimPostingDateId"], format="%Y-%m-%d")

## Create product info table

In [8]:
# Select only product relalted features
product_info = data[['ItemNo2','Color','Gender','Category','SUB Category', 'SUB Category2', 'Quality', 'Origin Country']].drop_duplicates()
product_info.reset_index(drop=True, inplace=True)
product_info

,ItemNo2,Color,Gender,Category,SUB Category,SUB Category2,Quality,Origin Country
0,F15404308_CLR000021,Black,WOMENSWEAR,Trousers,Trousers,Default,Woven,China
1,M00012003_CLR000508,White mel,MENSWEAR,Tops,Crew neck,Short sleeve,Jersey,Turkey
2,F19123672_CLR000021,Black,WOMENSWEAR,Shirts,Shirts,Short sleeve,Woven,China
3,F19123672_CLR000289,Clear Cream,WOMENSWEAR,Shirts,Shirts,Short sleeve,Woven,China
4,M00012307_CLR000650,Blue Iris Mel,MENSWEAR,Tops,V-neck,Short sleeve,Jersey,Turkey
...,...,...,...,...,...,...,...,...
695,M24400088_190414TCX,FOREST NIGHT,MENSWEAR,Shirts,Shirts,Default,Woven,China
696,F24200090_CLR000021,Black,WOMENSWEAR,Skirts,Maxi,Default,Woven,China
697,M24300006_CLR000589,Black Blizzard,MENSWEAR,Jeans,Jeans,Default,Denim,Turkey
698,F24400186_CLR000021,Black,WOMENSWEAR,Knitwear,Cardigan,Long sleeve,Knit,China


## Sample by week

In [9]:
# Sample weekly data
data_id_time_target = data.set_index("DimPostingDateId").groupby('ItemNo2').resample('W')['Quantity'].sum().reset_index()
data_id_time_target

,ItemNo2,DimPostingDateId,Quantity
0,F00001015_CLR000021,2022-03-20,66.0
1,F00001015_CLR000021,2022-03-27,92.0
2,F00001015_CLR000021,2022-04-03,90.0
3,F00001015_CLR000021,2022-04-10,70.0
4,F00001015_CLR000021,2022-04-17,78.0
...,...,...,...
56821,S00029_CLR001336,2024-08-11,0.0
56822,S00029_CLR001336,2024-08-18,4.0
56823,S00029_CLR001336,2024-08-25,2.0
56824,S00029_CLR001336,2024-09-01,1.0


## Calc MAE for naive forecast

In [10]:
def calc_mae_by_product_id(product_id, df):
    # Select records by product_id
    product_data = df.loc[df["ItemNo2"] == product_id]
    
    # Check if contains enough records
    if product_data.shape[0] < 16:
        return
    
    forecast = product_data.iloc[-16:-8]
    forecast.reset_index(drop=True, inplace=True)
    target = product_data.iloc[-8:]
    target.reset_index(drop=True, inplace=True)

    # Calc MAE
    result = (abs(target["Quantity"] - forecast["Quantity"])).mean()
    return result


In [11]:
# Apply mae calculation and store in a new column
product_info["naive_mae"] = product_info["ItemNo2"].apply(calc_mae_by_product_id, args=(data_id_time_target,))

## Calc MAE for forecasts

In [12]:
# Debug. Used for quickly review
def quick_check_by_product_id(unique_id):
    print("df_train:\n", df_train.loc[df_train["unique_id"] == unique_id].reset_index(drop=True))
    print("\ndf_test:\n", df_test.loc[df_test["unique_id"] == unique_id].reset_index(drop=True))
    print("\ndf_forecast:\n", df_forecast.loc[df_forecast["unique_id"] == unique_id].reset_index(drop=True))

In [13]:
def calc_mae_by_product_id(unique_id, df_forecast, df_test):
    # Select records from forecast data
    forecast_data = df_forecast.loc[df_forecast["unique_id"] == unique_id].reset_index(drop=True)
    if forecast_data.empty:
        return
    
    # Select records from test data
    target_data = df_test.loc[df_test["unique_id"] == unique_id].reset_index(drop=True)

    # Calc MAE
    result = (abs(target_data["y"] - forecast_data["forecast"])).mean()
    return result

In [14]:
# Apply mae calculation and store in a new column
product_info["forecast_mae"] = product_info["ItemNo2"].apply(calc_mae_by_product_id, args=(df_forecast,df_test))

In [15]:
product_info.head(20)

,ItemNo2,Color,Gender,Category,SUB Category,SUB Category2,Quality,Origin Country,naive_mae,forecast_mae
0,F15404308_CLR000021,Black,WOMENSWEAR,Trousers,Trousers,Default,Woven,China,38.625,41.250
1,M00012003_CLR000508,White mel,MENSWEAR,Tops,Crew neck,Short sleeve,Jersey,Turkey,26.750,15.500
2,F19123672_CLR000021,Black,WOMENSWEAR,Shirts,Shirts,Short sleeve,Woven,China,65.750,56.500
3,F19123672_CLR000289,Clear Cream,WOMENSWEAR,Shirts,Shirts,Short sleeve,Woven,China,66.250,74.000
4,M00012307_CLR000650,Blue Iris Mel,MENSWEAR,Tops,V-neck,Short sleeve,Jersey,Turkey,1.375,6.250
5,F15301618_CLR000289,Clear Cream,WOMENSWEAR,Tops,Tank,Sleeveless,Jersey,China,11.875,10.875
6,F00013006_CLR999254,3173 BLUE STRIPE,WOMENSWEAR,Tops,Tank,Long sleeve,Jersey,Turkey,1.500,0.000
7,M20300010_161103TCX,Pure Cashmere,MENSWEAR,Tops,Crew neck,Short sleeve,Jersey,Turkey,28.375,23.750
8,F00012030_CLR000021,Black,WOMENSWEAR,Tops,V-neck,Short sleeve,Woven,China,4.625,0.000
9,M20300010_CLR000337,Grey mel,MENSWEAR,Tops,Crew neck,Short sleeve,Jersey,Turkey,59.875,3.000


## Result interpretation


In [21]:
# Calc mean and std
product_info[['naive_mae', 'forecast_mae']].aggregate(['mean', 'std'])

,naive_mae,forecast_mae
mean,25.271976,17.258358
std,33.518968,30.577899


### Explanation of Results

The comparison of the mean and standard deviation of the `naive_mae` and `forecast_mae` demonstrates that our forecast model provides more accurate and consistent predictions compared to the naive forecast:

1. **Mean (Average Error):**  
   The mean MAE for the forecast model is noticeably lower than that of the naive forecast, indicating that the forecast model produces smaller errors on average and is therefore more accurate.

2. **Standard Deviation (Error Consistency):**  
   The standard deviation of the forecast model's errors is lower than that of the naive forecast, suggesting that the forecast model's predictions are more consistent and less prone to large deviations.

### Conclusion

These results indicate that the forecast model is effectively learning and leveraging patterns in the data to provide better forecasts than the naive approach, both in terms of accuracy and reliability. This highlights the model's ability to outperform a baseline method and underscores its value in improving forecast quality.